In [28]:
import pandas as pd
import numpy as np


In [29]:
from sklearn.preprocessing import LabelEncoder

ratings = pd.read_csv('ratings.csv', parse_dates=['timestamp'])

encoder1 = LabelEncoder()
encoder2 = LabelEncoder()

ratings['user_id'] = encoder1.fit_transform(ratings['user_id'])
ratings['product_id'] = encoder2.fit_transform(ratings['product_id'])
rand_user_ids = np.random.choice(ratings['user_id'].unique(), 
                                size=int(len(ratings['user_id'].unique())*0.3), 
                                replace=False)
ratings = ratings.loc[ratings['user_id'].isin(rand_user_ids)]

print('There are {} rows of data from {} users'.format(len(ratings), len(rand_user_ids)))
ratings.head()

There are 219587 rows of data from 12747 users


,user_id,product_id,rating,timestamp
6,40396,3893,1.0,1396051200
8,35978,3893,4.0,1362268800
13,40335,3893,5.0,1402876800
14,9330,3893,5.0,1360713600
17,15824,3893,5.0,1380326400


In [30]:
ratings['rank_latest'] = ratings.groupby(['user_id'])['timestamp'].rank(method='first', ascending=False)

train_ratings = ratings[ratings['rank_latest'] != 1]
test_ratings = ratings[ratings['rank_latest'] == 1]

# drop columns that we no longer need
train_ratings = train_ratings[['user_id', 'product_id', 'rating']]
test_ratings = test_ratings[['user_id', 'product_id', 'rating']]

In [31]:
train_ratings.loc[:, 'rating'] = 1
train_ratings.sample(5)

,user_id,product_id,rating
430196,639,40602,1
419003,40658,6839,1
691590,16664,27299,1
538367,22885,9628,1
3175,36229,23179,1


In [32]:
# Get a list of all Product IDs
all_product_ids = ratings['product_id'].unique()

# Placeholders that will hold the training data
users, items, labels = [], [], []

# This is the set of items that each user has interaction with
user_item_set = set(zip(train_ratings['user_id'], train_ratings['product_id']))

# 4:1 ratio of negative to positive samples
num_negatives = 4

for (u, i) in user_item_set:
    users.append(u)
    items.append(i)
    labels.append(1) # items that the user has interacted with are positive
    for _ in range(num_negatives):
        # randomly select an item
        negative_item = np.random.choice(all_product_ids) 
        # check that the user has not interacted with this item
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_product_ids)
        users.append(u)
        items.append(negative_item)
        labels.append(0) # items not interacted with are negative

In [33]:
import torch
from torch.utils.data import Dataset


class AmazonRatingsTrainDataset(Dataset):

    def __init__(self, ratings, all_product_ids):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_product_ids)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_product_ids):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['user_id'], ratings['product_id']))

        num_negatives = 4
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_product_ids)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_product_ids)
                users.append(u)
                items.append(negative_item)
                labels.append(0)
        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

In [34]:
class AmazonRatingsValDataset(Dataset):

    def __init__(self, ratings, all_product_ids):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_product_ids)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_product_ids):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['user_id'], ratings['product_id']))
        
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
        
        # create negative samples for validation data
        num_negatives = 99
        for u in set(ratings['user_id']):
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_product_ids)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_product_ids)
                users.append(u)
                items.append(negative_item)
                labels.append(0)
                
        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)
    
class AmazonRatingsTestDataset(Dataset):

    def __init__(self, ratings, all_product_ids):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_product_ids)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_product_ids):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['user_id'], ratings['product_id']))
        
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
        
        # create negative samples for test data
        num_negatives = 99
        for u in set(ratings['user_id']):
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_product_ids)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_product_ids)
                users.append(u)
                items.append(negative_item)
                labels.append(0)
                
        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)


In [35]:
import torch.nn as nn
import torch
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import DataLoader
from pytorch_lightning import loggers as pl_loggers
import sklearn
import sklearn.metrics
# ...
import pytorch_lightning as pl

# replace: from pytorch_lightning.metrics import functional as FM
# with the one below
import torchmetrics

# import lightning_flash, which we’ll use later
import flash
from flash.image import ImageClassifier, ImageClassificationData

class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the product ratings for training
            all_product_ids (list): List containing all product_id (train + test)
    """
    
    def __init__(self, num_users, num_items, ratings, all_product_ids):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_product_ids = all_product_ids
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        
        #loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())    
        loss_fn = torch.nn.BCELoss()
        loss = loss_fn(torch.sigmoid(predicted_labels), labels.float())
        preds = torch.sigmoid(predicted_labels) > 0.5
        acc = (preds == labels).float().mean()
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        logits = self(user_input, item_input)
        loss_fn = torch.nn.BCELoss()
        val_loss = loss_fn(torch.sigmoid(logits), labels.float())
        preds = torch.sigmoid(logits) > 0.5
        val_acc = (preds == labels).float().mean()
        self.log('val_loss', val_loss, on_step=True, on_epoch=True)
        self.log('val_acc', val_acc, on_step=True, on_epoch=True)
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(AmazonRatingsTrainDataset(self.ratings, self.all_product_ids),
                          batch_size=512, num_workers=10)

In [36]:
import torch.nn as nn
import torch
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import DataLoader
from pytorch_lightning import loggers as pl_loggers
import sklearn
import sklearn.metrics
# ...
import pytorch_lightning as pl

# replace: from pytorch_lightning.metrics import functional as FM
# with the one below
import torchmetrics

# import lightning_flash, which we’ll use later
import flash
from flash.image import ImageClassifier, ImageClassificationData

class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the product ratings for training
            all_product_ids (list): List containing all product_id (train + test)
    """
    
    def __init__(self, num_users, num_items, ratings, all_product_ids):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_product_ids = all_product_ids
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        
        # reshape labels tensor to have the same shape as predicted labels
        labels = labels.view(-1, 1).float()
        
        loss_fn = torch.nn.BCELoss()
        loss = loss_fn(torch.sigmoid(predicted_labels), labels.float())
        preds = torch.sigmoid(predicted_labels) > 0.5
        acc = (preds == labels).float().mean()
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        logits = self(user_input, item_input)
        # reshape labels tensor to have the same shape as predicted labels
        labels = labels.view(-1, 1).float()
        loss_fn = torch.nn.BCELoss()
        val_loss = loss_fn(torch.sigmoid(logits), labels.float())
        preds = torch.sigmoid(logits) > 0.5
        val_acc = (preds == labels).float().mean()
        self.log('val_loss', val_loss)
        self.log('val_acc', val_acc)
        
    def train_dataloader(self):
        return DataLoader(AmazonRatingsTrainDataset(self.ratings, self.all_product_ids),
                          batch_size=512, num_workers=0)
    
    def val_dataloader(self):
        return DataLoader(AmazonRatingsValDataset(self.ratings, self.all_product_ids),
                          batch_size=512, num_workers=0)

    def test_dataloader(self):
        return DataLoader(AmazonRatingsTestDataset(self.ratings, self.all_product_ids),
                          batch_size=512, num_workers=10)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())


In [37]:
#num_users = ratings['user_id'].max()
num_users = ratings['user_id'].max()+1
num_items = ratings['product_id'].max()+1
#num_items = ratings['product_id'].max()+1
all_product_ids = ratings['product_id'].unique()

model = NCF(num_users, num_items, train_ratings, all_product_ids)



In [38]:
logger = pl_loggers.TensorBoardLogger('logs/')
trainer = pl.Trainer(max_epochs=3,logger=logger, accelerator='auto', val_check_interval=1.0)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [39]:
trainer.fit(model)


  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 339 K 
1 | item_embedding | Embedding | 440 K 
2 | fc1            | Linear    | 1.1 K 
3 | fc2            | Linear    | 2.1 K 
4 | output         | Linear    | 33    
---------------------------------------------
783 K     Trainable params
0         Non-trainable params
783 K     Total params
3.136     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/suyashnehete/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/suyashnehete/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [40]:
trainer.logged_metrics

{'train_loss': tensor(0.6931),
 'train_acc': tensor(0.5720),
 'val_loss': tensor(0.6931),
 'val_acc': tensor(0.5911)}

In [45]:
import matplotlib.pyplot as plt
ep = [20]
bs = [128]


criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())
# Instantiate the training and validation datasets and data loaders
train_dataset = AmazonRatingsTrainDataset(train_ratings, all_product_ids)
val_dataset = AmazonRatingsValDataset(test_ratings, all_product_ids)
for num_epochs, batch_size in zip(ep, bs):
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Train the PyTorch model
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    hit_rate = []
    for epoch in range(num_epochs):
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        for batch_idx, (user_ids, item_ids, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(user_ids, item_ids)
            loss = criterion(outputs, labels.float().view(-1, 1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            # Calculate training accuracy
            predicted = outputs > 0.5
            train_correct += (predicted == labels.float().view(-1, 1)).sum().item()
            train_total += len(labels)

        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        train_acc = train_correct / train_total
        train_accuracies.append(train_acc)

        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for batch_idx, (user_ids, item_ids, labels) in enumerate(val_loader):
                outputs = model(user_ids, item_ids)
                loss = criterion(outputs, labels.float().view(-1, 1))
                val_loss += loss.item()

                # Calculate validation accuracy
                predicted = outputs > 0.5
                val_correct += (predicted == labels.float().view(-1, 1)).sum().item()
                val_total += len(labels)

            val_loss /= len(val_loader)
            val_losses.append(val_loss)
            val_acc = val_correct / val_total
            val_accuracies.append(val_acc)
        
        # User-item pairs for testing
        test_user_item_set = set(zip(test_ratings['user_id'], test_ratings['product_id']))

        # Dict of all items that are interacted with by each user
        user_interacted_items = ratings.groupby('user_id')['product_id'].apply(list).to_dict()

        hits = []
        for (u,i) in test_user_item_set:
            interacted_items = user_interacted_items[u]
            not_interacted_items = set(all_product_ids) - set(interacted_items)
            selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
            test_items = selected_not_interacted + [i]
            predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                                torch.tensor(test_items)).detach().numpy())

            top25_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:25].tolist()]
            if i in top25_items:
                hits.append(1)
            else:
                hits.append(0)
        hit_rate.append(np.average(hits))

        print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}, Val Loss: {:.4f}, Val Acc: {:.4f}, Hits: {:.4f}'.format(
            epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc, np.average(hits)))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    plt.savefig(f'cb/ep {num_epochs} bs {batch_size} tl vl.png')

    plt.plot(train_accuracies, label='Training Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.plot(hit_rate, label='Hit Rate')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    plt.savefig(f'cb/ep {num_epochs} bs {batch_size} ta va.png')
    



Epoch [1/20], Train Loss: 0.3078, Train Acc: 0.8570, Val Loss: 0.2026, Val Acc: 0.9255, Hits: 0.5783
Epoch [2/20], Train Loss: 0.2969, Train Acc: 0.8641, Val Loss: 0.2076, Val Acc: 0.9207, Hits: 0.5715


KeyboardInterrupt: 

In [ ]:
# Plot the training and validation loss curves
import matplotlib.pyplot as plt
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss/Accuracy')
plt.legend()
plt.show()

plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss/Accuracy')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_loss(train_losses, val_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training loss')
    plt.plot(val_losses, label='Validation loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss over Epochs')
    plt.legend()
    plt.show()

def plot_accuracy(train_accs, val_accs):
    plt.figure(figsize=(10, 5))
    plt.plot(train_accs, label='Training accuracy')
    plt.plot(val_accs, label='Validation accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy over Epochs')
    plt.legend()
    plt.show()

def plot_confusion_matrix(cm, classes, title):
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

def plot_roc_curve(fpr, tpr, roc_auc):
    plt.figure(figsize=(8, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

def plot_precision_recall(precision, recall, average_precision):
    plt.figure(figsize=(8, 8))
    plt.step(recall, precision, where='post', label='Precision-Recall curve (AP = %0.2f)' % average_precision)
    plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower right")
    plt.show()


In [34]:

# User-item pairs for testing
test_user_item_set = set(zip(test_ratings['user_id'], test_ratings['product_id']))

# Dict of all items that are interacted with by each user
user_interacted_items = ratings.groupby('user_id')['product_id'].apply(list).to_dict()

hits = []
for (u,i) in test_user_item_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_product_ids) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top25_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:25].tolist()]
    if i in top25_items:
        hits.append(1)
    else:
        hits.append(0)
        
print("The Hit Ratio @ 25 is {:.2f}".format(np.average(hits)))

The Hit Ratio @ 25 is 0.61


In [22]:
u = 205424347
if u not in ratings['user_id'].unique():
    print('no')
else:
    interacted_items = user_interacted_items[u]
    not_interacted_items = list(set(all_product_ids) - set(interacted_items))
    predicted_labels = np.squeeze(model(torch.tensor([u]*len(not_interacted_items)), torch.tensor(not_interacted_items)).detach().numpy())
    items = [not_interacted_items[i] for i in np.argsort(predicted_labels)[::-1][0:100].tolist()]
    items

no


In [75]:
test_user_item_set

{(20547, 37727),
 (27766, 36806),
 (8843, 741),
 (24778, 12907),
 (3184, 26286),
 (18206, 37061),
 (26541, 15784),
 (11386, 51957),
 (39189, 8481),
 (1959, 34577),
 (2861, 18170),
 (35603, 27266),
 (5215, 4253),
 (42040, 44574),
 (39408, 33142),
 (25477, 53188),
 (17327, 4295),
 (28888, 37785),
 (38492, 3076),
 (28218, 49102),
 (28633, 3035),
 (32622, 13916),
 (7736, 40663),
 (39771, 36577),
 (28848, 52881),
 (12312, 8967),
 (3823, 38408),
 (6079, 28615),
 (1458, 10095),
 (23569, 15436),
 (8428, 53493),
 (21904, 32315),
 (19064, 46783),
 (29629, 24584),
 (36261, 40899),
 (4650, 6551),
 (2521, 51848),
 (25534, 43751),
 (29615, 43375),
 (914, 34962),
 (16958, 44442),
 (2161, 38215),
 (19729, 19028),
 (1601, 18401),
 (14734, 54941),
 (19697, 50981),
 (30838, 47439),
 (19258, 54317),
 (4682, 45874),
 (20984, 450),
 (28506, 1516),
 (25827, 2575),
 (3957, 329),
 (31776, 47170),
 (13019, 31288),
 (10192, 33681),
 (39729, 31749),
 (36364, 52223),
 (10294, 47958),
 (17319, 52153),
 (34945, 3463

In [76]:
[20547]*100

[20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547,
 20547]